In [1]:
import pandas as pd
import numpy as np
import typing as tp

In [2]:
from tg.grammar_ru import Separator


text = (
    'Это предложение для того, чтобы построить бандл. '
    'Это предложение не для этого. ' 
    'Еще одно предложение, в котором есть запятая. '
    'В этом предложении, как можно заметить, целых две запятые. '
    'В этом предложении есть новый знак препинания: двоеточие. '
    'Тире - еще один знак препинания. '
    ', А вот это предложение начинается с запятой, правда ведь неожиданно?'
)
df = Separator.separate_string(text)
df

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length
0,0,0,0,0,1,Это,ru,3
1,1,0,1,0,1,предложение,ru,11
2,2,0,2,0,1,для,ru,3
3,3,0,3,0,0,того,ru,4
4,4,0,4,0,1,",",punct,1
...,...,...,...,...,...,...,...,...
61,61,6,8,0,1,",",punct,1
62,62,6,9,0,1,правда,ru,6
63,63,6,10,0,1,ведь,ru,4
64,64,6,11,0,0,неожиданно,ru,10


In [3]:
from tg.common import DataBundle
from tg.grammar_ru.common import Loc

db = DataBundle.load(Loc.bundles_path/'punct/140k')

In [4]:
from tg.common.ml import batched_training as bt

idb = bt.IndexedDataBundle(
    index_frame = db.index.sample(5, random_state=45), 
    bundle = db
)
idb.index_frame

,word_id,sentence_id,label,reference,target_word,split
sample_id,,,,,,
30508,30508,30492,0,lenta.base.zip/30492,",",train
112497,112497,112299,0,pub.base.zip/112299,no,test
73177,73177,73145,0,lenta.base.zip/73145,no,train
55515,55515,55503,0,lenta.base.zip/55503,no,train
17783,17783,17761,0,lenta.base.zip/17761,no,test


In [50]:
from tg.common.ml.batched_training import context as btc
from tg.common.ml import batched_training as bt
from tg.grammar_ru.components import PlainContextBuilder
import pandas as pd
import numpy as np


class PunctContextBuilder(PlainContextBuilder):
    def build_context(self, ibundle: bt.IndexedDataBundle, context_size) -> pd.DataFrame:
        context_df = super().build_context(ibundle, context_size)
        context_df = context_df.reset_index()
        word_is_target = (
            ibundle.bundle.src
            .set_index('word_id')
            .loc[context_df.sample_id].is_target
            .values
        )
        max_offset = context_df.offset.max()
        rows_to_drop = (~(word_is_target & (context_df.offset == 1))) & ~(~word_is_target & (context_df.offset == max_offset))
        context_df = context_df[rows_to_drop]

        return context_df.set_index(['sample_id', 'offset'])


In [51]:
context_builder = PunctContextBuilder(
    include_zero_offset=True,
    left_to_right_contexts_proportion=0.5
)

context_df = context_builder.build_context(idb, 5).sort_index()
context_df

another_word_id
sample_id offset                 
17783     -2                17781
          -1                17782
           0                17783
           1                17784
30508     -2                30506
          -1                30507
           0                30508
           2                30510
55515     -2                55513
          -1                55514
           0                55515
           1                55516
73177     -2                73175
          -1                73176
           0                73177
           1                73178
112497    -2               112495
          -1               112496
           0               112497
           1               112498